### Twilight Struggle

In [2]:
import numpy as np
import random

from twilight_map import *
from game_mechanics import *
from twilight_cards import *

IndentationError: unindent does not match any outer indentation level (game_mechanics.py, line 867)

### Game Instance

In [ ]:
# this is a temporary measure. It should go into UI.new()
g = Game()
g.ui.run()